Enter your username (used for marking):

In [1]:
username = 'myusername'

# COM4509/6509 Coursework Part 1

Hello,
This is the first of the two parts. Each part accounts for 50\% of the overall coursework mark.

### What to submit

- You need to submit **two jupyter notebooks** (not zipped together), named:

```
assignment_part1_[username].ipynb
assignment_part2_[username].ipynb
```

replacing `[username]` with your username, e.g. `abc18de`.

- Please do not upload the data files used in this Notebook. We just want the two python notebooks.

### Assessment Criteria 

- The marks are indicated for each part: You'll get marks for correct code that does what is asked and gets the right answer. These contribute 45.
- There are also 5 marks for "Code quality" (includes both readability and efficiency).

### Late submissions

We follow the department's guidelines about late submissions, Undergraduate [handbook link](https://sites.google.com/sheffield.ac.uk/comughandbook/your-study/assessment/late-submission). PGT [handbook link](https://sites.google.com/sheffield.ac.uk/compgtstudenthandbook/home/your-study/assessment/late-submission).

### Use of unfair means
"Any form of unfair means is treated as a serious academic offence and action may be taken under the Discipline Regulations." (from the students Handbook).

# A dataset of air quality

We are going to use a dataset of air pollution measurements in Beijing archived by the UCI. To read about the dataset visit the [UCI page](https://archive.ics.uci.edu/ml/datasets/Beijing+Multi-Site+Air-Quality+Data).

We are going to:
  1. Preprocess the data
  2. Build our own Lasso-regression
  3. Use sklearn's tools to perform regression on the data
  
Let's get started

In [2]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import urllib
import matplotlib.pyplot as plt
%matplotlib inline

We will be trying to predict the pollution (PM2.5) using:
- temperature ('TEMP')
- pressure ('PRES')
- dew-point temperature ('DEWP')
- precipitation ('RAIN')
- wind direction ('wd')
- wind speed ('WSPM')

In [3]:
urllib.request.urlretrieve('https://drive.google.com/uc?id=1m1g4Xn1wMAGV_EU0Nh1HTI1ogA3-tqJk&export=download', './data.csv')

('./data.csv', <http.client.HTTPMessage at 0x7fa3f859bca0>)

In [4]:
raw_df = pd.read_csv('data.csv',index_col='No')

#put the columns in a useful order
raw_df = raw_df[['PM2.5', 'year', 'month', 'day', 'hour', 'PM10', 'SO2', 'NO2', 'CO',
       'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'wd', 'WSPM', 'station']]

Some of the records are missing. We need to handle that before we can easily use the data with most ML tools.

### Question 1: Removing missing data [1 mark]

We are going to handle this by dropping those rows which have a NaN in one of these columns: ['PM2.5','hour','TEMP','PRES','DEWP','RAIN','wd','WSPM'].

Save the result in `nonull_df`.

We can use `df.dropna` to do this. Pandas documentation on this method is [here](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html).

In [5]:
#Put answer here
nonull_df = raw_df.dropna(subset=['PM2.5','hour','TEMP','PRES','DEWP','RAIN','wd','WSPM'])

To check you've done it correctly, you could use `clean_df.isnull().sum()` to confirm that there are no NaN rows in the columns we're interested in:

In [6]:
nonull_df.isnull().sum()

PM2.5        0
year         0
month        0
day          0
hour         0
PM10        17
SO2        242
NO2        331
CO         866
O3         685
TEMP         0
PRES         0
DEWP         0
RAIN         0
wd           0
WSPM         0
station      0
dtype: int64

### Question 2: Removing unwanted columns [1 mark]

Let's remove the columns we're not going to be using. We can use `nonull_df.drop(list_of_column_names, axis=1)` to do this. We will drop: ['year','month','day','PM10','SO2','NO2','CO','O3','station'].

Again, feel free to check if it's worked with `clean_df.isnull().sum()`, for example.

In [7]:
clean_df = nonull_df.drop(['year','month','day','PM10','SO2','NO2','CO','O3','station'], axis=1)
clean_df.isnull().sum()

PM2.5    0
hour     0
TEMP     0
PRES     0
DEWP     0
RAIN     0
wd       0
WSPM     0
dtype: int64

In [8]:
#there should be 34284 rows left in your dataframe, and 8 columns (note, this was corrected on 7/11/22)
clean_df.shape #=(34284, 8)

(34284, 8)

### Question 3: Splitting the dataset [2 marks]

Before designing any machine learning model, we need to set aside the test data. We will use the remaining training data for fitting the model. **It is important to remember that the test data has to be set aside before preprocessing**.

Any preprocessing that you do has to be done only on the training data and several key statistics need to be saved for the test stage. Separating the dataset into training and test before any preprocessing has happened help us to recreate the real world scenario where we will deploy our system and for which the data will come without any preprocessing.

Later we will be performing a grid search to select parameter values. To do this we'll do cross-validation, but rather than split the data into training and validation here we'll split it later. So for now we'll just split into:

- The training (and validation) set will have 85% of the total observations, 
- The test set, the remaining 15%.

To avoid unwanted correlations connecting the training and test, we will split these by time. So:

- Take the first 85% of the rows from clean_df and put them in train_df, take the remaining 15% of the rows and put them in test_df

In [9]:
#Put answer here
# Split the data without reshuffling because otherwise test data samples could be 
# very similar to train data particularly at a given hour

split = 0.85
idxSplit = int(split * len(clean_df))

train_df = clean_df.iloc[:idxSplit, :]
test_df = clean_df.iloc[idxSplit:, :]

To check the sizes are correct, we can use:

In [10]:
len(train_df)/len(clean_df),len(test_df)/len(clean_df)

(0.8499883327499709, 0.15001166725002918)

# Detour: Lasso Regression

Later we will use the sklearn toolkit, but in this section **you will develop your own code** to do the Lasso regression.

### Ordinary Least Squares Regression

First, let's just perform normal linear regression.

We'll use a toy design matrix & labels to use to check our code works. We'll also specify a weight vector too, for testing.

In [11]:
X = np.array([[0.0,0],[1,3],[2.2,3]])
y = np.array([0.0,1,2])
w = np.array([1.0,2])

### Question 4: Prediction Function [2 marks]

The first task is to write a function to make predictions. Can you complete this function for linear regression, i.e. the predictions for all our points $f(X,\mathbf{w}) = X \mathbf{w}$. [corrected: 7/11/22, don't need $X$ tranposed]

In [12]:
def prediction(X,w):
    return X @ w

In [13]:
#You can use this code to check you've written the right function
np.all(prediction(X,w)==np.array([0. , 7. , 8.2])) #Should return 'True'

True

### Question 5: Objective Function [4 marks]

Now we need to write a function that returns the 'error'.
We'll just do normal Ordinary Least Squares with linear regression, so if you remember the cost function for that is:

$$E = \sum_{i=1}^N \big(y_i-f(\mathbf{x}_i,\mathbf{w}) \big)^2$$

Where $E$ is the error, $N$ the number of points, $y_i$ is one of the labels, $\mathbf{x}_i$ is the input for that label, $\mathbf{w}$ is the weight vector. $f$ is the prediction function you've already written. Or feel free to substitute in $\mathbf{x}_i^\top \mathbf{w}$.

In [14]:
def objective(X,y,w):
    """
    Computes the sum squared error (for us to perform OLS linear regression).
    """
    return np.sum(np.square(y - X @ w))

In [15]:
#You can use this code to check you've written the right function
objective(X,y,w)==74.44

True

### Question 6: Objective Function Gradient [4 marks]

Now you need to find the derivative of the objective wrt the parameter vector. You've already done this in lectures, so remember the gradient of the error function (for linear regression) is:

$$\frac{\partial E}{\partial \mathbf{w}} = 2 X^\top X \mathbf{w} - 2 X^\top y$$

Add code to do this here:

In [16]:
def objective_derivative(X,y,w):
    """
    Computes the derivative of the sum squared error, wrt the parameters.
    """
    return 2 * X.T @ X @ w - 2 * X.T @ y 
objective_derivative(X,y,w)

array([39.28, 73.2 ])

To check your gradients are correct, we can estimate the gradient numerically:

In [17]:
def numerical_objective_derivative(X,y,w):
    """
    Computes a numerical approximation to the derivative of the sum squared error, wrt the parameters.
    """
    g = np.zeros_like(w)
    for i,wi in enumerate(w):
        d = np.zeros_like(w)
        d[i]=1e-6
        g[i] = (objective(X,y,w+d)-objective(X,y,w-d))/2e-6
    return g

The two gradient vectors should be approximately equal:

In [18]:
objective_derivative(X,y,w),numerical_objective_derivative(X,y,w)

(array([39.28, 73.2 ]), array([39.28, 73.2 ]))

### Question 7: Optimise $\mathbf{w}$ to minimise the objective [4 marks]

Now you need to use the gradient function you've written to maximise $\mathbf{w}$ using gradient descent.
Start with a sensible choice of w. You'll need to loop lots of times (e.g. 1000). At each iteration: compute the gradient and subtract the scaled gradient from the w parameter (you'll need to scale it by the learning rate, of e.g. 0.01).

In [19]:
def optimise_parameters(X,y,startw):
    """
    Returns the w that minimises the objective.
    """
    lr = 1e-2
    w = startw
    for i in range(1000):
        
        loss = objective(X, y, w)
        
        if i%100==0: print(f"Iteration {i}, Loss: {loss:.3e}")
        
        w -= lr * objective_derivative(X, y, w)
        
    return w

In [20]:
bestw = optimise_parameters(X,y,w)
print(bestw) #print our solution

Iteration 0, Loss: 7.444e+01
Iteration 100, Loss: 3.716e-02
Iteration 200, Loss: 3.960e-03
Iteration 300, Loss: 4.220e-04
Iteration 400, Loss: 4.498e-05
Iteration 500, Loss: 4.793e-06
Iteration 600, Loss: 5.108e-07
Iteration 700, Loss: 5.443e-08
Iteration 800, Loss: 5.801e-09
Iteration 900, Loss: 6.182e-10
[0.8333238 0.0555608]


Let's compare this to the answer provided by sklearn:

In [21]:
from sklearn import linear_model
clf = linear_model.LinearRegression(fit_intercept=False)
clf.fit(X,y)
print(clf.coef_) #matches the value of w we found above, hopefully!

[0.83333333 0.05555556]


## Lasso Regression

### Question 8: New Objective Function [3 marks]

We're now going to regularise the regression using $L_1$ regularisation - i.e. Lasso regression.

We need a **new objective function**:

$$E = \frac{1}{2N}\sum_{i=1}^N \big(y_i-f(\mathbf{x}_i,\mathbf{w}) \big)^2 + \alpha \sum_{j=1}^P |w_j|$$

This is similar to before (but the first term is now half the mean squared error, rather than the sum squared error). The second term is the L1 regularisation term.

In [22]:
def objective_lasso(X,y,w,alpha):
    """
    Computes half the mean squared error, with an additional L1 regularising term. alpha controls the level of regularisation.
    """
    return 0.5 * np.mean(np.square(y - X @ w)) + alpha * np.sum(np.abs(w))

### Question 9: The gradient of the lasso regression objective [3 marks]

The tricky bit the derivative of the objective.

The first part is similar to before. So, with the regularising term, the derivative is:
$$\frac{\partial E}{\partial \mathbf{w}} = \frac{1}{N}(X^\top X \mathbf{w} - X^\top y) + \alpha\;\text{sign}(\mathbf{w})$$

where $\text{sign}(\mathbf{w})$ returns a vector of the same shape as $\mathbf{w}$ with +1 if the element is positive and -1 if it's negative. The `np.sign` method does this for you.

Have a think about why this is (think about what differentiating the 'absolute' function $|w_j|$ involves - think about what happens when it's positive vs when it's negative. 

In [23]:
def objective_lasso_derivative(X,y,w,alpha):
    """
    Returns the derivative of the Lasso objective function.
    """
    N = len(y)
    return 1/N * (X.T @ X @ w - X.T @ y) + alpha * np.sign(w)

We can check it again, numerically. The two pairs of parameters should be the same:

In [24]:
def numerical_objective_lasso_derivative(X,y,w,alpha):
    """
    This finds a numerical approximation to the true gradient
    """
    g = np.zeros_like(w)
    for i,wi in enumerate(w):
        d = np.zeros_like(w)
        d[i]=1e-6
        g[i] = (objective_lasso(X,y,w+d,alpha)-objective_lasso(X,y,w-d,alpha))/2e-6
    return g

objective_lasso_derivative(X,y,w,0.1),numerical_objective_lasso_derivative(X,y,w,0.1)

(array([0.09999823, 0.10000097]), array([0.09999823, 0.10000097]))

### Question 10: Optimise $\mathbf{w}$ to minimise the Lasso objective [2 marks]

As before we need to optimise to find the optimum value of $\mathbf{w}$, for this Lasso objective.
You'll need to loop lots of times (e.g. 5000). Start with a sensible choice of w. At each iteration: compute the gradient and subtract the scaled gradient from the w parameter (you'll need to scale it by the learning rate, of e.g. 0.05).

In [25]:
def optimise_parameters_lasso(X,y,startw):
    """
    Returns the w that minimises the Lasso objective.
    """
    #Put answer here
    lr = 5e-2
    w = startw
    alpha = 0.1
    
    for i in range(5000):
        
        loss = objective_lasso(X, y, w, alpha)
        
        if i%100==0: print(f"Iteration {i}, Loss: {loss:.3e}")
        
        w -= lr * objective_lasso_derivative(X, y, w, alpha)
        
    return w

optimise_parameters_lasso(X,y,w)

Iteration 0, Loss: 8.889e-02
Iteration 100, Loss: 8.417e-02
Iteration 200, Loss: 8.362e-02
Iteration 300, Loss: 8.353e-02
Iteration 400, Loss: 8.352e-02
Iteration 500, Loss: 8.352e-02
Iteration 600, Loss: 8.352e-02
Iteration 700, Loss: 8.352e-02
Iteration 800, Loss: 8.352e-02
Iteration 900, Loss: 8.352e-02
Iteration 1000, Loss: 8.352e-02
Iteration 1100, Loss: 8.352e-02
Iteration 1200, Loss: 8.352e-02
Iteration 1300, Loss: 8.352e-02
Iteration 1400, Loss: 8.352e-02
Iteration 1500, Loss: 8.352e-02
Iteration 1600, Loss: 8.352e-02
Iteration 1700, Loss: 8.352e-02
Iteration 1800, Loss: 8.352e-02
Iteration 1900, Loss: 8.352e-02
Iteration 2000, Loss: 8.352e-02
Iteration 2100, Loss: 8.352e-02
Iteration 2200, Loss: 8.352e-02
Iteration 2300, Loss: 8.352e-02
Iteration 2400, Loss: 8.352e-02
Iteration 2500, Loss: 8.352e-02
Iteration 2600, Loss: 8.352e-02
Iteration 2700, Loss: 8.352e-02
Iteration 2800, Loss: 8.352e-02
Iteration 2900, Loss: 8.352e-02
Iteration 3000, Loss: 8.352e-02
Iteration 3100, Loss

array([0.63888889, 0.14259259])

We can check against the sklearn method:

In [26]:
clf = linear_model.Lasso(alpha=0.1,fit_intercept=False)
clf.fit(X,y)
print(clf.coef_)

[0.63931263 0.1423666 ]


The above result should approximately match the one you computed.

# Back to air pollution

### Question 11: One-hot-encoding [4 marks]

One of the columns isn't numerical, but instead is a string type: The wind direction. The best way to deal with this is one-hot-encoding.

pandas has a tool for doing this: `pd.get_dummies(series, prefix='prefix_to_use')`. In our example the series is: `clean_df.wd`.

You'll need to:
1. Make the one-hot encoding table using the code above.
2. Delete the `wd` column from our table (hint: you did this earlier for other columns).
3. Join the one hot data to the table. To do this use something like `dataframe1.join(dataframe2)`.



In [27]:
def add_wd_onehot(df):
    """Add new one-hot encoding set of columns, removes the old column it's made from. Returns new dataframe."""
    
    # Get one hot encoding of columns 'vehicleType'
    table = pd.get_dummies(df["wd"])

    # Drop column as it is now encoded
    encoded_df = df.drop(["wd"], axis=1)

    # Join the encoded df
    encoded_df = encoded_df.join(table)

    return encoded_df

#you could use this code to see if it's worked?
train_df_wdencoded = add_wd_onehot(train_df)
train_df_wdencoded

,PM2.5,hour,TEMP,PRES,DEWP,RAIN,WSPM,E,ENE,ESE,...,NNW,NW,S,SE,SSE,SSW,SW,W,WNW,WSW
No,,,,,,,,,,,,,,,,,,,,,
1,9.0,0,0.3,1021.9,-19.0,0.0,2.0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,11.0,1,-0.1,1022.4,-19.3,0.0,4.4,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,8.0,2,-0.6,1022.6,-19.7,0.0,4.7,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,8.0,3,-0.7,1023.5,-20.9,0.0,2.6,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5,8.0,4,-0.9,1024.1,-21.7,0.0,2.5,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29760,138.0,23,28.2,997.0,24.2,0.0,1.0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
29761,145.0,0,27.7,997.1,24.3,0.0,0.9,0,0,0,...,0,0,0,0,0,1,0,0,0,0
29762,139.0,1,26.9,996.6,24.6,0.0,0.8,1,0,0,...,0,0,0,0,0,0,0,0,0,0


### Question 12: Standardise the data [3 marks]
[note: updated from term 'Normalise' to 'Standardise' on 7/11/22. For clarity, I want the mean to be zero and the standard deviation to be one].

Now we need to standardise [edit: corrected] the data.

You could manipulate just some columns by using, for example: `df.iloc[:,1:]` - this returns a dataframe that consists of all but the first column.

Feel free to use either tools from `sklearn.preprocessing` or standardise [edit: corrected on 7/11/22] it by, for example, using the mean and the standard deviation of the columns by calling `some_dataframe.mean()` or `some_dataframe.std()`.

In [50]:
def standardise(df, train_df):
    """
    Returns a new dataframe in which all but the PM2.5 columns are standardised (i.e. have a mean of zero and standard deviation of 1)
    [note: the function name used to be 'normalise' but was modified for clarity]

    [addition: 7/11/22
    Think about if you want to standardise using the *training* data's mean and standard deviation (for the test data).
    """
    
    # Use training set to standardize df, both for training and test
    stand_df = df.copy()
    stand_df.iloc[:, 1:] -= train_df.iloc[:,1:].mean()
    stand_df.iloc[:, 1:] /= train_df.iloc[:, 1:].std()
    
    return stand_df

train_df_preprocessed = standardise(add_wd_onehot(train_df), add_wd_onehot(train_df))

# I believe the preprocessing on 
test_df_preprocessed = standardise(add_wd_onehot(test_df), add_wd_onehot(train_df))

# Check first column is unchanged and rest of columns are standardized'
train_df_preprocessed.std(), test_df_preprocessed.std()

# And we check training data is standardized to 1 while test data has more variance around 1
# This is becuase the test data preprocessing is using the training data means and std

(PM2.5    82.432774
 hour      1.000000
 TEMP      1.000000
 PRES      1.000000
 DEWP      1.000000
 RAIN      1.000000
 WSPM      1.000000
 E         1.000000
 ENE       1.000000
 ESE       1.000000
 N         1.000000
 NE        1.000000
 NNE       1.000000
 NNW       1.000000
 NW        1.000000
 S         1.000000
 SE        1.000000
 SSE       1.000000
 SSW       1.000000
 SW        1.000000
 W         1.000000
 WNW       1.000000
 WSW       1.000000
 dtype: float64,
 PM2.5    103.261131
 hour       0.997833
 TEMP       1.044087
 PRES       0.942879
 DEWP       1.051096
 RAIN       1.128614
 WSPM       0.891056
 E          1.030425
 ENE        1.172341
 ESE        1.101838
 N          1.137986
 NE         1.392286
 NNE        1.137867
 NNW        1.186881
 NW         1.027553
 S          0.914140
 SE         1.297810
 SSE        1.047106
 SSW        0.711625
 SW         0.934248
 W          0.536410
 WNW        0.522378
 WSW        0.686292
 dtype: float64)

Here we put the training and test inputs (X) and outputs (y) into four variables:

In [29]:
X = train_df_preprocessed.iloc[:,1:].to_numpy()
y = train_df_preprocessed.iloc[:,0].to_numpy()

Xtest = test_df_preprocessed.iloc[:,1:].to_numpy()
ytest = test_df_preprocessed.iloc[:,0].to_numpy()

We can use the same code we wrote before, using the Lasso from sklearn to fit the data. Here we'll turn fit_intercept on, as we've not added a '1's column to our design matrix.

So feel free to use:
```
clf = linear_model.Lasso(alpha=0.1,fit_intercept=True)
clf.fit(X,y)
```

### Question 13: Finding the RMSE of the Lasso regressor predictions [2 marks]

Next compute the **RMSE** of the predictions for (a) the training data and (b) the test data.
The RMSE (root mean squared error) could be computed, for example with:

```np.sqrt(np.mean((predicted_values-true_values)**2))```

In [30]:
def RMSE(ypred, y):
    return np.sqrt(np.mean((ypred - y)**2))

In [31]:
# Specify model
clf = linear_model.Lasso(alpha=0.1, fit_intercept=True)

# Run optimization to find minimum
clf.fit(X, y)

# Make prediction using found minima
ypred_l_train = clf.predict(X)
ypred_l_test = clf.predict(Xtest)

# Errors
rmse_lasso_train = RMSE(ypred_l_train, y)
rmse_lasso_test = RMSE(ypred_l_test, ytest) 

# RMSE uses the same formula as the std but instead of comparing values to
# the mean, the errors are computed on a point by point basis
rmse_lasso_train, rmse_lasso_test

(72.40451940025356, 87.60477726176146)

We can compare this to the standard deviation of the data, we should do better than that!

In [32]:
np.std(y), np.std(ytest)

(82.43136001277581, 103.25109103178704)

### Question 14: Random Forest [8 marks]

The final step is to use a random forest regressor.

If we use the default random forest regressor, we find we get considerable over-fitting. So we need to explore different parameters. We will use a cross-validated grid search over the parameters:

- max_features: The number of features to consider when looking for the best split (i.e. controls subsampling), *from 1 to the number of features* in 4 steps (e.g. use `np.linspace`)
- n_estimators: The number of trees in the forest, from 10 to 100 in 4 steps.
- max_samples : the number of samples to draw from to train each base estimator, from 0.1 to 0.9 in 4 steps.

We will use `GridSearchCV`.

Have a look at the documentation for this, the three parameters we need to specify are:

- the 'estimator': an **INSTANCE** of RandomForestRegressor.
- param_grid: a **DICTIONARY**, each item is the title of the parameter, and equals an array of the values we need to test. For exmaple, one of the items might be `{'max_samples': np.linspace(0.1, 0.9, 5)}`.
- You'll need to think carefully how to make the lists for the `max_features` and `n_estimators` as these both need to be (positive) integers. E.g. use `.astype(int)`.

In [33]:
Ncols = len(X[0])
assert len(Xtest[0])==Ncols, "Number of columns for Xtest and X need to be the same"

#1. Create a grid of parameter values for n_estimators, max_features and max_samples,
grid_pars = {
    "n_estimators" : np.linspace(10, 100, 4).astype(int),      # Number of trees
    "max_features" : np.linspace(1, Ncols, 4).astype(int),     # Number of columns to pick for each tree
    "max_samples" : np.linspace(0.1, 0.9, 4)       # Bootstrap set to True by default
}
#2. Create a GridSearchCV object, using the random forest regressor:
grid_regression = GridSearchCV(RandomForestRegressor(), param_grid=grid_pars)

#Note: Because there is so much training data, using the full dataset takes too long. So here we'll just use 10%
np.random.seed(42)
idx = np.sort(np.random.choice(len(X), size=int(len(X)*0.1), replace=False))
#3. Fit to training data in (the subset of) X and y
grid_regression.fit(X[idx,:],y[idx])

GridSearchCV(estimator=RandomForestRegressor(),
             param_grid={'max_features': array([ 1,  8, 15, 22]),
                         'max_samples': array([0.1       , 0.36666667, 0.63333333, 0.9       ]),
                         'n_estimators': array([ 10,  40,  70, 100])})

Here we print the best parameters from the grid search (on the training/validation cross-validation run):

In [34]:
# Print best values
for key in grid_regression.best_params_:
    print(f"Best {key} : {grid_regression.best_params_[key]}")

Best max_features : 8
Best max_samples : 0.6333333333333333
Best n_estimators : 40


### Question 15: RMSE for the Random Forest Regressor [1 mark]

Finally compute the RMSE for the training and test data:

In [35]:
# Create new Forest with best found parameters
bestForest = RandomForestRegressor(**grid_regression.best_params_)

# Run Random Forest
bestForest.fit(X, y)

# Make predictions
ypred_rf_train = bestForest.predict(X)
ypred_rf_test = bestForest.predict(Xtest)

# Errors
rmse_rf_train = RMSE(ypred_rf_train, y) 
rmse_rf_test = RMSE(ypred_rf_test, ytest)

rmse_rf_train, rmse_rf_test

(31.366585598892463, 76.23356023972379)

In [36]:
# Actually, I found out that using a brand new forest and
# the predict() method of grid_regression object yields different results.
# Not sure why this is, since from the documentation the grid_regression object
# should use the same model and the same best found parameters.

for obj, name in zip([bestForest, grid_regression], ["Best forest", "Grid regression"]):
    train_err = RMSE(obj.predict(X), y)
    test_err = RMSE(obj.predict(Xtest), ytest)
    print(f"\n{name} RMSE:\ntrain: {train_err:.3f}, test: {test_err:.3f}")
    
# From these results, it seems that the best approach is the best forest one,
# i.e. start with a new RandomForestRegressor() explicitly with the best parameters
# from grid_regression


Best forest RMSE:
train: 31.367, test: 76.234

Grid regression RMSE:
train: 64.083, test: 79.858


We can compare this to the standard deviations for the two sets of data.

In [37]:
np.std(y), np.std(ytest)

(82.43136001277581, 103.25109103178704)

In [38]:
rmse_lasso_train, rmse_lasso_test

(72.40451940025356, 87.60477726176146)

### Question 16: Did the random forest do better than lasso regression? [1 mark]



In [39]:
# The RMSE of the random forest (took the best forest method) is lower than 
# in the lasso regression, however this differnce is more noticeable 
# in the training set rather than in the test set.
# Still, for the test set, the RMSE of the random forest is lower than lasso regression,
# so random forest did perform better.